# 6.2 Logistic Regression Pipeline

##### Description

Pipelines to train the models are implemented in this notebook. The preprocessing steps of scaling and encoding are embedded in the pipeline. The exploratory models are trained to determine which has the highest accuracy score.

##### Notebook Steps

1. Connect Spark
1. Input data
1. Basic data review
1. Visualize relationships

## 1. Connect Spark

In [1]:
%load_ext sparkmagic.magics

In [2]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

Added endpoint http://ec2-52-23-187-221.compute-1.amazonaws.com:8998/
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1611655226728_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [3]:
%%spark
spark.sparkContext.setCheckpointDir('./checkpoints')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 2. Load Data
There are two seperate datasets we will be working with while modeling. They are train.csv and validate.csv. Train is used to train the model, while validate is used to perform evaluation on unseen data. Only train is needed at this time.

In [4]:
%%spark
train = spark.read.csv("s3://jolfr-capstone3/training/train.csv", header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
%%spark

train, drop = train.randomSplit(weights = [0.25, 0.75], seed = 42)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 4. Define Pipeline Steps

 1. Feature Hasher
 1. Standard Scaler
 1. Model

Collects column names for later use in pipeline.

In [6]:
%%spark
cols = train.drop("label").columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Define Feature Hasher

In [7]:
%%spark
from pyspark.ml.feature import FeatureHasher
hasher = FeatureHasher(inputCols=cols, outputCol="hash")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Define Scaler

In [8]:
%%spark
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol=hasher.getOutputCol(), outputCol="features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 5. Define Model and Parameter Grid

In [9]:
%%spark
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder

lr = LogisticRegression(maxIter=10)

params = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]).addGrid(lr.elasticNetParam, [0.4, 0.8]).build()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 6. Define Pipeline

In [10]:
%%spark
from pyspark.ml import Pipeline
pipe = Pipeline(stages=[hasher, scaler, lr])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 6. Define Cross Validator

In [11]:
%%spark
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
crossval = CrossValidator(estimator=pipe, estimatorParamMaps=params, evaluator=BinaryClassificationEvaluator(), numFolds=5, parallelism=5, seed=42)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 7. Fit Model

In [12]:
%%spark
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col
train = train.withColumn("label", col("label").cast(FloatType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
%%spark
cvModel = crossval.fit(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 8. Save Best Params

In [ ]:
%%spark
best_params = cvModel.bestModel.stages[-1].extractParamMap()

params = {}

for k, v in best_params.items():
    params[k.name] = v
    
params

In [ ]:
%%spark
import boto3
import pickle

s3 = boto3.client('s3')

serialized = pickle.dumps(params)

s3.put_object(Bucket='jolfr-capstone3',Key='LogisticParams', Body=serialized)

In [ ]:
%%spark
import boto3
import pickle

s3 = boto3.client('s3')

serialized = pickle.dumps(params)

s3.put_object(Bucket='jolfr-capstone3',Key='LogisticParams', Body=serialized)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'aggregationDepth': 2, 'elasticNetParam': 0.4, 'family': 'auto', 'featuresCol': 'features', 'fitIntercept': True, 'labelCol': 'label', 'maxIter': 10, 'predictionCol': 'prediction', 'probabilityCol': 'probability', 'rawPredictionCol': 'rawPrediction', 'regParam': 0.01, 'standardization': True, 'threshold': 0.5, 'tol': 1e-06}

In [36]:
%%spark
import boto3
import pickle

s3 = boto3.client('s3')

serialized = pickle.dumps(params)

s3.put_object(Bucket='jolfr-capstone3',Key='LogisticParams', Body=serialized)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '7B4809D5C6A8DCE6', 'HostId': 'ugvWN7XwOMU8Z+6yMxNuPhpoQGZocpHZLOq+zvVWNE/r+2TX4VK+jts8SZ01RQh7poERjqKImH0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'ugvWN7XwOMU8Z+6yMxNuPhpoQGZocpHZLOq+zvVWNE/r+2TX4VK+jts8SZ01RQh7poERjqKImH0=', 'x-amz-request-id': '7B4809D5C6A8DCE6', 'date': 'Tue, 26 Jan 2021 12:21:36 GMT', 'etag': '"7477ce1c985d791efffd18db09fd746a"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"7477ce1c985d791efffd18db09fd746a"'}